# Redis Assignment

The purpose of this task is to create the cookie syncronization service between two companies with the help of Redis.

Your code will be tested by calling the combination of functions:
* `set_ttls`
* `save_sync`
* `get_uid`
* `get_partner_uid`
* `limit_rate`

Hints:

1. Do not forget that your `get_partner_uid` and `get_uid` functions should return decoded value if the ttl is not exired.  If ttl expires,  `get_partner_uid` and `get_uid` functions must return None

2. Make sure that the function `save_sync` returns `0` as default if the `ttl` for the partner is not specified

3. The function `limit_rate` must be used in both `get_partner_uid` and `get_uid` and keyname should be f"hit:{function.__name__}:{partner_id}:{current_time_in_seconds}" 

4. The function `limit_rate` must raise `LimitExceededException` if the number of calls for one partner in the last second is greater than `MAX_RPS`

5. You must use the global variable `MAX_RPS` in `limit_rate` as the maximum value of the usage amount

If you want to validate your code by yourself then you can find `time.sleep(seconds)` helpful

In [1]:
from typing import Union, Callable
import redis
import time


class LimitExceededException(Exception):
    pass


MAX_RPS = 5 # requests per second
r = redis.StrictRedis(host='localhost', port=6381, db=0)

In [15]:
def save_sync(r: redis.StrictRedis, uid: str, partner_id: int, partner_uid: str):
    """Set the values for the pairs <partner_id, uid> and <partner_id, partner_uid>
 
    Do not forget to set the ttls which you defined in the function set_ttls
    
    Agrs:
        r (redis.StrictRedis): redis instance
        uid (str): cookie uid
        partner_id (int): id of the partner
        partner_uid (str): uid of the partner
  
    Examples:
        >>> save_sync(r, 'uid_1', 10, 'partner_uid_1')
    
    """
    ttl = r.hget("ttls", partner_id) or 0
    
    r.set("{}:{}".format(partner_id,uid),partner_uid)
    r.expire("{}:{}".format(partner_id,uid),ttl)
    r.set("{}:{}".format(partner_id,partner_uid),uid)
    r.expire("{}:{}".format(partner_id,partner_uid),ttl)
    


def limit_rate(r: redis.StrictRedis, function: Callable, partner_id: int):
    """Restrict function usage by MAX_RPS requests per second.
    
    If the amount of function calls is greater than MAX_RPS, raise LimitExceededException(f"{MAX_RPS} limit  is reached").
   
    Args:
        r (redis.StrictRedis): redis instance
        function (Callable): function from which limit_rate is called
        partner_id (int): id of the partner
       
    Examples:
        >>> limit_rate(r, get_partner_uid, partner_id)
        
    """
    key = 'hit:{}:{}:{}'.format(function.__name__, partner_id, int(time.time()))
    r.incr(key)
    rps = int(r.get(key))
    if rps > MAX_RPS:
        raise LimitExceededException("{} limit  is reached".format(MAX_RPS))


def get_partner_uid(r: redis.StrictRedis, uid: str, partner_id: int):
    """Get the partner id by the pair (uid, partner id)
    
    Args:
        r (redis.StrictRedis): redis instance
        uid (str): cookie uid
        partner_id (int): id of the partner
        
    Examples:
        >>> get_partner_uid(r, 'e5a370cc-6bdc-43ae-baaa-8fd4531847f7', 12)
 
    """
    limit_rate(r, get_partner_uid, partner_id)
    puid = r.get("{}:{}".format(partner_id,uid))
    if puid:
        puid = puid.decode('utf-8')
    return puid


def get_uid(r: redis.StrictRedis, partner_id: int, partner_uid: str):
    """Get the uid by the pair (partner id, partner uid)
    
    Args:
        r (redis.StrictRedis): redis instance
        partner_id (int): id of partner
        partner_uid (str): uid of partner
        
    Examples:
        >>> get_uid(r, 12, '25b6e9a6-fca8-427c-94df-2577e62b2bf0')
 
    """
    limit_rate(r, get_uid, partner_id)
    uid = r.get("{}:{}".format(partner_id,partner_uid))
    if uid:
        uid = uid.decode('utf-8')
    return uid


def set_ttls(r: redis.StrictRedis, ttls: dict):
    """Set the ttl by partner id
      
    Args:
        r (redis.StrictRedis): redis instance
        ttls (dict): dictionary of pairs <partner_id, ttl>
        
    Examples:
        >>> set_ttls(r, {12: 5, 3: 1})
 
    """
    for key,val in ttls.items():
        r.hset("ttls",key,val)
    

In [16]:
# usage examples

set_ttls(r, {12: 5, 3: 0.25})

save_sync(r, '365402ea-1942-4dc5-a70b-c40467b49e39', 12, '4b6b3c9e-82f4-48a7-a87e-8f9e856fe303')

get_partner_uid(r, '365402ea-1942-4dc5-a70b-c40467b49e39', 12)
# should return
# '4b6b3c9e-82f4-48a7-a87e-8f9e856fe303'

get_uid(r, 12, '4b6b3c9e-82f4-48a7-a87e-8f9e856fe303')
# should return
# '365402ea-1942-4dc5-a70b-c40467b49e39'

'365402ea-1942-4dc5-a70b-c40467b49e39'

In [17]:
# Place for testing your code